In [1]:
# Imports

import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
import tqdm as tqdm
from butifarra_env import ButifarraEnv
import pettingzoo
from collections import defaultdict
import supersuit
import ray.rllib.algorithms.ppo as ppo
import stable_baselines3 as stable
%matplotlib inline

2023-11-30 10:47:18,956	WARNING __init__.py:10 -- PG has/have been moved to `rllib_contrib` and will no longer be maintained by the RLlib team. You can still use it/them normally inside RLlib util Ray 2.8, but from Ray 2.9 on, all `rllib_contrib` algorithms will no longer be part of the core repo, and will therefore have to be installed separately with pinned dependencies for e.g. ray[rllib] and other packages! See https://github.com/ray-project/ray/tree/master/rllib_contrib#rllib-contrib for more information on the RLlib contrib effort.


In [2]:
# pettingzoo to stable baselines
env = ButifarraEnv(action_mask_on_obs=True, bad_action_reward=0)
from ray.tune.registry import register_env
from ray.rllib.env import PettingZooEnv
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.examples.policy.random_policy import RandomPolicy
from ray.rllib.examples.models.shared_weights_model import (
    SharedWeightsModel1,
    SharedWeightsModel2,
)
from ray.rllib.examples.models.action_mask_model import TorchActionMaskModel, ActionMaskModel
from ray.rllib.models import ModelCatalog
from ray.rllib.algorithms.callbacks import DefaultCallbacks
import ray.rllib.examples.models.action_mask_model
print(ray.rllib.examples.models.action_mask_model.__file__)

register_env("butifarra", lambda config: PettingZooEnv(env))
ModelCatalog.register_custom_model(
    "am_model",
    TorchActionMaskModel
)

class MyCallback(DefaultCallbacks):
    def on_episode_end(self, worker, base_env, policies, episode,
                **kwargs):
        for (agent_id, agent_policy), agent_rewards in episode.agent_rewards.items():
            # raise NotImplementedError(f"{agent_id=} {agent_rewards=}")
            episode.custom_metrics[agent_id + '_reward'] = agent_rewards


policies = {
    "random": (RandomPolicy, env.observation_spaces["player_0"], env.action_spaces["player_0"], {}),
    "trained": (None, env.observation_spaces["player_1"], env.action_spaces["player_1"], {}),
}


def policy_chooser(agent_id, episode, **kwargs):
    if agent_id in ["player_0", "player_2"]:
        return "random"
    else:
        return "trained"

config = ppo.PPOConfig()
config.environment("butifarra")
config.framework("torch")
config.multi_agent(policies=policies, policy_mapping_fn=policy_chooser, policies_to_train=["trained"])
config.reporting()
config.training(model={"custom_model": "am_model",
                       "custom_model_config": {
                           "mask_invalid_actions": True,
                       }})
config.training(_enable_learner_api=False)._enable_rl_module_api = False
config.callbacks(MyCallback)
# config = {
#     "env": "butifarra",
#     "framework": "torch",
#     "model": {
#         "custom_model": "am_model",
#     },
#     "multiagent": {
#         "policies": policies,
#         "policy_mapping_fn": policy_chooser,
#         "policies_to_train": ["player_1", "player_3"]
#     },
#     "reporting": {},
#     "training": {
#         "_enable_learner_api": False,
#     },
#     "rl_modules": {
#         "_enable_rl_module_api": False,
#     },
# }


/home/inigo/.local/lib/python3.8/site-packages/ray/rllib/examples/models/action_mask_model.py


/home/inigo/.local/lib/python3.8/site-packages/pettingzoo/utils/wrappers/base.py:58: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  warnings.warn(
/home/inigo/.local/lib/python3.8/site-packages/pettingzoo/utils/wrappers/base.py:72: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn(


In [3]:
from ray import air, tune
results = tune.Tuner(
    "PPO", param_space=config, run_config=air.RunConfig(stop={"training_iteration": 10000}, verbose=1)
).fit()


(pid=377084) 2023-11-30 10:47:25,253	WARNING __init__.py:10 -- PG has/have been moved to `rllib_contrib` and will no longer be maintained by the RLlib team. You can still use it/them normally inside RLlib util Ray 2.8, but from Ray 2.9 on, all `rllib_contrib` algorithms will no longer be part of the core repo, and will therefore have to be installed separately with pinned dependencies for e.g. ray[rllib] and other packages! See https://github.com/ray-project/ray/tree/master/rllib_contrib#rllib-contrib for more information on the RLlib contrib effort.
(RolloutWorker pid=377150) /home/inigo/.local/lib/python3.8/site-packages/pettingzoo/utils/wrappers/base.py:58: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
(RolloutWorker pid=377150)   warnings.warn(
(RolloutWorker pid=377150) /home/inigo/.local/lib/python3.8/site-packages/pettingzoo/utils/wrappers/base.py:72: UserWarning: The `action_spaces` dictionary is deprecated. Use th